<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introcción

Con el siguiente algoritmo presentado se busca determinar cual fue el año en el que se procesaron la mayor cantidad de compras por parte del Estado Nacional de Argentina, estos datos son provistos de la pagina oficial del estado[1].
Para esto se utilizara el la programacion paralela por el gran caudal de datos. Se utilizara la interfaz de programación paralela OpenMP.
Con la ejecucion paralela disminuimos en gran medida los largos tiempos de procesamientos en esta clase de archivos con extensa longitud de elementos a procesar, procesaremos los diferentes archivos por separado para poder realizar comparaciones entre los resultados y asi poder determinar quien fue el año de mayor compras.

#2. Armado del ambiente

Es necesario descargar los archivos que seran utilizados por el algoritmo

In [ ]:
url_archivo_1 = "https://github.com/StankoDiego/SOA_EA_2/blob/main/2014.txt?raw=true"
url_archivo_2 = "https://github.com/StankoDiego/SOA_EA_2/blob/main/2015.txt?raw=true"
url_archivo_3 = "https://github.com/StankoDiego/SOA_EA_2/blob/main/2016.txt?raw=true"
url_archivo_4 = "https://github.com/StankoDiego/SOA_EA_2/blob/main/2017.txt?raw=true"
url_archivo_5 = "https://github.com/StankoDiego/SOA_EA_2/blob/main/2018.txt?raw=true"
url_archivo_6 = "https://github.com/StankoDiego/SOA_EA_2/blob/main/2019.txt?raw=true"
url_archivo_7 = "https://github.com/StankoDiego/SOA_EA_2/blob/main/2020.txt?raw=true"

!wget {url_archivo_1} -O "2014.txt"
!wget {url_archivo_2} -O "2015.txt"
!wget {url_archivo_3} -O "2016.txt"
!wget {url_archivo_4} -O "2017.txt"
!wget {url_archivo_5} -O "2018.txt"
!wget {url_archivo_6} -O "2019.txt"
!wget {url_archivo_7} -O "2020.txt"

#3. Desarrollo

In [ ]:
code = """
// Procesamiento de archivos con OpenMP, usando c, ejecutado en Colab. 

#include <iostream>
#include <vector>
#include <cstdlib>
#include <fstream>
#include <string>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP   

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_AXPY_SEC      2
#define HTH_AXPY_OMP      3

// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
  int i;
  TIEMPO_INI( HTH_TOTAL )

  if( argc < 2)
  {
      std::cerr<< " Error en los parametros de indicar: (cant_archivos), archivos (20XX.txt)."<<argc<<std::endl;
      exit( -1 );
  }

  int cant_files = atoi(argv[1]);
  int number_of_lines[cant_files];

  int j = 0;
  for(j = 0; j < cant_files; j++){
    number_of_lines[j] = 0;
  }
   TIEMPO_INI( HTH_AXPY_OMP )   

    #pragma omp parallel for
    for(i=0; i < cant_files; i++){
      number_of_lines[i] = 0;
      std::string line;
      std::ifstream myfile(argv[2 + i]);
      while (std::getline(myfile, line)){
        number_of_lines[i]++;
      }
      std::cout << "Nombre de archivo: " << argv[2 + i];
      std::cout << " " << std::endl;
      std::cout << "Lineas de texto de archivo: " << number_of_lines[i];
      std::cout << " " << std::endl;
    }
    
    TIEMPO_FIN( HTH_AXPY_OMP )
    TIEMPO_FIN( HTH_TOTAL )

    int max = number_of_lines[0];
    int pos = -1;

    for(i = 1; i < cant_files; i++){
      if(max < number_of_lines[i]){
        max = number_of_lines[i];
        pos = i;
      }
    }

 std::cout<<"--------------------------------------------------------"<<std::endl;
 std::cout << "Nombre de archivo: " << argv[pos + 2]<<std::endl;
 std::cout << "Maxima cantidad de lineas de texto de archivo: " << number_of_lines[pos]<<std::endl;
 std::cout<<"--------------------------------------------------------"<<std::endl;
 std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
 std::cout<<"Tiempo Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;
 std::cout<<std::endl;

}
"""
text_file = open("archivos.cpp", "w")
text_file.write(code)
text_file.close()

##3.1 Compilacion de código C Archivo

In [ ]:
!g++ -o archivos -fopenmp archivos.cpp

##3.2 Ejecucion de ejecutable Archivo 

In [ ]:
%env OMP_NUM_THREADS=10
!./archivos 7 "2014.txt" "2015.txt" "2016.txt" "2017.txt" "2018.txt" "2019.txt" "2020.txt" 

env: OMP_NUM_THREADS=10
Nombre de archivo: 2015.txt 
Lineas de texto de archivo: 44724 
Nombre de archivo: 2014.txt 
Lineas de texto de archivo: 30326 
Nombre de archivo: 2019.txt 
Lineas de texto de archivo: 36819 
Nombre de archivo: 2020.txt 
Lineas de texto de archivo: 29518 
Nombre de archivo: 2018.txt 
Lineas de texto de archivo: 42225 
Nombre de archivo: 2017.txt 
Lineas de texto de archivo: 42225 
Nombre de archivo: 2016.txt 
Lineas de texto de archivo: 43704 
--------------------------------------------------------
Nombre de archivo: 2015.txt
Maxima cantidad de lineas de texto de archivo: 44724
--------------------------------------------------------
Tiempo TOTAL     : 22.3269 [ms]
Tiempo Omp  : 22.326 [ms]



#4. Tabla de pasos

Tabla de ejecucion de openMP
>Lenguaje|Procesador | Funcion | Detalle
>--- | --- | --- |---
>Phyton|CPU	|code|Codigo Python que define el código C para implementar openMP
>Phyton|CPU|text_file = open()|Genera el archivo .cpp para escritura
>Phyton|CPU|text_file.write|Escribe el archivo .cpp
>Phyton|CPU|text_file.close|Cierra el archivo .cpp
>C|CPU|TIEMPO_INI( HTH_TOTAL )|Toma el tiempo inicial de la ejecucion
>C|	CPU|TIEMPO_INI( HTH_AXPY_OMP )|Toma el tiempo inicial del procesamiento paralelo
>C|**GPU**|#pragma omp parallel for|Se define el fragemento de código que se ejecutara de forma paraleal
>C|**GPU**|for(i=0; i < cant_files; i++)|Ejecuta de forma paralela el codigo interno del for
>C|	CPU|TIEMPO_FIN( HTH_AXPY_OMP )|Toma el tiempo final del procesamiento paralelo
>C|	CPU|TIEMPO_FIN( HTH_TOTAL )|Toma el tiempo final de la ejecucion del programa
>C|	CPU|TIEMPO_FIN( HTH_TOTAL )|Toma el tiempo final de la ejecucion del programa
>C|	CPU|std::cout<<""<<std::endl|Muestra de información
>bash | CPU | g++ | Compilacion de programa .cpp
>bash | CPU | !./Archivos | Ejecucion del programa

    

#5. Conclusiones

Se opto por la utilizacion de OpenMP para resolver el algoritmo ya que permite crear hilos de ejecucion de forma simple, esto es necesario ya que los archivos procesados poseen un basto nivel de informacion. Se desperdiciaría un tiempo excesivo en procesar el archivo de forma secuencial. El codigo que genera los diferentes hilos es programado en C, sin la necesidad de la libreas pthread.h. El programa se alimenta de diferentes archivos (fue necesario separlarlo en diferentes archivos ya que no permitia ser subido de forma completo) y cada uno se procesara en un hilo de ejecucion diferente. Para continuar con el ejercicio se podría realizar estadisticas sobre las empresas que vendieron mas productos al estado y por que provincias fueron realizadas.

#6. Recursos utilizados

[1].Datos.gob.ar [Archivo usado](https://datos.gob.ar/dataset/energia-refinacion-comercializacion-petroleo-gas-derivados-tablas-dinamicas/archivo/energia_6282dbf1-93d7-4fd3-a63a-61a8f8a95475)

